*   Author : Ziang Xu
*   Student number : 180104048
*   Code : U-net model construction, training and testing.

In [0]:
# Google Colaboratory carry Google drive.
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# Enter the segmentation task directory.
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/ML/segmentation/")
!ls

In [0]:
# Model building.
import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.optimizers import SGD
K.set_image_data_format('channels_last')  # TF dimension ordering in this code

img_rows = 224
img_cols = 224

smooth = 1
def get_unet(pretrained_weights=None):
    '''
    Set Input layer for 224x224

    '''
    inputs = Input((img_rows, img_cols,1))
    '''
    Down-sampling path.Each block consists 2 conv layers and 1 maxpooling layer.
    
    '''
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    '''
    Up-simpling path and skip connection.    
  
    '''
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-4), loss =dice_coef_loss , metrics = [dice_coef])

    if (pretrained_weights):
        model.load_weights(pretrained_weights)

    return model


In [0]:
model=get_unet()

In [0]:
# Custom loss function.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [0]:
# Store the training dataset in .npy format for easy recall.
import os
import numpy as np
from skimage.io import imsave, imread
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
data_path = 'data/' # Setting path
image_rows = 512
image_cols = 512
def create_train_data():
    train_data_path = os.path.join(data_path, 'resize_train/Image') # Training file path
    train_data_Label_path = os.path.join(data_path, 'resize_train/Label') # Mask file path
    images = os.listdir(train_data_path)
    total = len(images)
    imgs = np.ndarray((total,image_rows, image_cols), dtype=np.uint8)
    imgs_mask = np.ndarray((total,image_rows, image_cols), dtype=np.uint8)
    i = 0
    print('Creating training images...')
    for image_name in images:
        img = imread(os.path.join(train_data_path, image_name), as_grey=True)
        img_mask = imread(os.path.join(train_data_Label_path, image_name), as_grey=True)
        
        img = img_to_array([img])
        img_mask = img_to_array([img_mask])
       
        imgs[i] = img
        imgs_mask[i] = img_mask
        if i % 100 == 0:
            print('Done: {0}/{1} images'.format(i, total))
        i += 1
    print('Loading done.')
    np.save('imgs_train.npy', imgs)
    np.save('imgs_mask_train.npy', imgs_mask)
    print('Saving to .npy files done.')

create_train_data()

In [0]:
import pandas as pd
pd.Series(imgs_mask_train[1].reshape((1,-1))[0]).describe()

In [0]:
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

In [0]:
# Read data
import numpy as np
imgs_train = np.load('imgs_train.npy')
imgs_mask_train = np.load('imgs_mask_train.npy')
imgs_train = preprocess(imgs_train)
imgs_mask_train = preprocess(imgs_mask_train)


In [0]:
# Normalized dataset
imgs_train = imgs_train.astype('float32')
imgs_train /=255.0
mean = np.mean(imgs_train)  # mean for data centering 
imgs_train -= mean
imgs_mask_train = imgs_mask_train.astype('float32')
imgs_mask_train /= 255.0 # scale masks to [0, 1]
model = get_unet()

In [0]:
# Setting checkpoint
model_checkpoint = ModelCheckpoint('unet_weights.h5', monitor='val_loss',verbose=1, save_best_only=True)

In [0]:
# Model training.
history=model.fit(imgs_train, imgs_mask_train, batch_size=20, nb_epoch=200, verbose=1, shuffle=True,
              validation_split=0.2,
              callbacks=[model_checkpoint])

In [0]:
# Draw a graph of accuracy and loss functions.
from matplotlib import pyplot as plt
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# Store the testing dataset in .npy format for easy recall.
import os
import numpy as np
from skimage.io import imsave, imread
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
data_path = 'data/'

def create_test_data():
    test_data_path = os.path.join(data_path, 'test_small/Image') #训练文件路径
   
    images = os.listdir(test_data_path)
    total = len(images)
    imgs = np.ndarray((total,img_rows, img_cols), dtype=np.uint8)
    
    i = 0
    print('Creating training images...')
    for image_name in images:
        img_id = int(image_name.split('.')[0])
        img = imread(os.path.join(test_data_path, image_name), as_grey=True)
        
        img = np.array([img])
        
        
       
        imgs[i] = img
        
        if i % 100 == 0:
            print('Done: {0}/{1} images'.format(i, total))
        i += 1
    print('Loading done.')
    np.save('imgs_test.npy', imgs)
    
    print('Saving to .npy files done.')

create_test_data()

In [0]:
# Loading dataset.
import numpy as np
imgs_test = np.load('imgs_test.npy')
# imgs_id_test = np.load('imgs_id_test.npy')

In [0]:
from keras.models import load_model
# model=load_model('unet_weights.h5')
model = get_unet('model/unet_weights.h5')

In [0]:
imgs_test = preprocess(imgs_test)
imgs_test = imgs_test.astype('float32')
imgs_test /= 255.

In [0]:
# Model prediction/.
imgs_mask_test = model.predict(imgs_test, verbose=1)

In [0]:
np.save('imgs_mask_test.npy', imgs_mask_test)
# imgs_mask_test = np.load('imgs_mask_test.npy')
imgs_id_test=np.load('imgs_id_test.npy')

In [0]:
# Convert test data from arrays to  mask images.
from data import *
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
def save_img():
		print("array to image")
		imgs = imgs_mask_test
		for i in range(imgs.shape[0]):
			img = imgs[i]
      
			img = array_to_img(img)
			img.save("data/results/results_UNET/%d.jpg"%(i))
save_img()